In [1]:
# Cell 1: Imports
import os
from dotenv import load_dotenv
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType

load_dotenv()

print("Libraries loaded successfully.")

c:\Anaconda3\envs\pashdoc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries loaded successfully.


In [2]:
# Cell 2: Define File and Initialize Loader
# REPLACE 'policy.pdf' with the name of your actual insurance file
file_path = "SBIhomeinsurance_home.pdf"

if not os.path.exists(file_path):
    print(f"⚠️ ERROR: File '{file_path}' not found. Please put your PDF in this folder.")
else:
    print(f"File found: {file_path}")
    
    # We use ExportType.MARKDOWN. 
    # This tells Docling: "Do not split this yet. Just convert the visual PDF to a structured Markdown string."
    # This is the best way to verify if tables were read correctly.
    loader = DoclingLoader(
        file_path=file_path,
        export_type=ExportType.MARKDOWN
    )
    
    print("Loader initialized. Ready to process...")

File found: SBIhomeinsurance_home.pdf
Loader initialized. Ready to process...


In [3]:
# Cell 3: Execute Loading (The Heavy Lifting)
# This step might take 10-30 seconds depending on your machine (it's running OCR/AI models locally).

print("Processing document... (This runs locally, please wait)")
docs = loader.load()

print(f"Processing complete!")
print(f"Generated {len(docs)} LangChain Document object(s).")

2025-12-10 17:57:22,549 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]


Processing document... (This runs locally, please wait)


2025-12-10 17:57:26,668 - INFO - Going to convert document batch...
2025-12-10 17:57:26,672 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
2025-12-10 17:57:26,757 - INFO - Loading plugin 'docling_defaults'
2025-12-10 17:57:26,768 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-12-10 17:57:26,770 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-10 17:57:26,846 - INFO - Loading plugin 'docling_defaults'
2025-12-10 17:57:26,866 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-12-10 17:57:26,868 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-12-10 17:57:29,320 - INFO - Accelerator device: 'cpu'
[INFO] 2025-12-10 17:57:29,433 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-

Processing complete!
Generated 1 LangChain Document object(s).


In [4]:
# Cell 4: Verify Metadata
# In production, we need to know WHERE this data came from.
# Docling automatically adds metadata (page numbers, source).

source_doc = docs[0]
print("Metadata Check:")
print(source_doc.metadata)

Metadata Check:
{'source': 'SBIhomeinsurance_home.pdf'}


In [35]:
# Cell 5:
# This is the most important step for Insurance RAG.
# We print a portion of the content to see if the tables look like Markdown tables.

print("-" * 60)
print("CONTENT PREVIEW (First 2000 characters):")
print("-" * 60)


content = source_doc.page_content
print(content[:1000])  #21000:30000

------------------------------------------------------------
CONTENT PREVIEW (First 2000 characters):
------------------------------------------------------------
## SIMPLE HOME INSURANCE POLICY POLICY WORDING

You chose this Simple Home Insurance Policy and applied to Us for insurance covers of Your choice. You paid Us the premium and gave Us  information  about  Yourself,  Your  Home  Building  and  Home Contents. Based on Your confirmation that this information is true and correct, and in return of accepting the Premium You have paid Us We  promise  to  provide  You  insurance  as  stated  in  this  Policy Document and the Policy Schedule attached to it.

1. Your Policy : This Simple Home Insurance Policy is a contract between You and Us as stated in the following:
- a. This  Policy  document,
- b. The Policy  Schedule  attached  to  this  Policy  document,
- c. Any Endorsement attached to and forming part of this Policy document,
- d.  Any Add-on to this Policy that You may have pu

In [ ]:
# Cell 6.1: The "Two-Step" Splitting Pipeline

from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

# --- STEP 1: Split by Structure (Logical) ---
# We first group text by headers to capture "Context"
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# This gives us chunks that might be HUGE, but they have good metadata
md_header_splits = markdown_splitter.split_text(source_doc.page_content)
print(f"Step 1 (Headers) produced {len(md_header_splits)} logical sections.")

Step 1 (Headers) produced 135 logical sections.


In [69]:
## just to check the count of each chunk in Header

doc0 = md_header_splits[1]
print(doc0.page_content)

print(len(doc0.page_content))



- a.  This Policy is issued to You and covers You and/or Your Home Building and/or Home Contents as mentioned in the Policy Schedule.
- b.  If more than one person is insured under this Policy, each of You is a joint policyholder. Any notice or letter We give to any of You will be considered as given to all of You. Any request, statement, representation, claim or action of any one of You will  bind  all  of  You  as  if  made  by  all  of  You.
- c. If You have mortgaged, pledged or hypothecated Your Home Building  and/or  Home  Contents  with  a  Bank,  the  Policy Schedule will  show  an  
'Agreed Bank Clause' and the name of such Bank. The terms and conditions of this arrangement will be added to this Policy as an  additional  clause.  
3. The  Policy  Schedule: The  Policy  Schedule  is  an  important document about Your insurance cover. It contains:
- a. Your personal details,
- b. the  Policy  Period,
- c. the  description  of  Your  Insured  Property,
- d. the total  Sum  Insure

In [43]:
# Cell 6.2: The "Two-Step" Splitting Pipeline

# --- STEP 2: Split by Size (Physical Constraint) ---
# Now we force these logical sections into strict size limits
chunk_size = 1000  # Characters (approx 200-250 words)
chunk_overlap = 200 # Keep some overlap so we don't cut sentences in half

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap
)

# Crucial: We pass the 'md_header_splits' (which are Documents), not raw text.
# This ensures the new small chunks INHERIT the metadata (Header 1, Header 2) from Step 1.
final_chunks = text_splitter.split_documents(md_header_splits)

print(f"Step 2 (Size) produced {len(final_chunks)} final chunks ready for embedding.")

Step 2 (Size) produced 212 final chunks ready for embedding.


In [70]:
final_chunks[1]

Document(metadata={'Header 2': '2. To  whom  this  Policy  is  issued  and  what  it  covers:'}, page_content="- a.  This Policy is issued to You and covers You and/or Your Home Building and/or Home Contents as mentioned in the Policy Schedule.\n- b.  If more than one person is insured under this Policy, each of You is a joint policyholder. Any notice or letter We give to any of You will be considered as given to all of You. Any request, statement, representation, claim or action of any one of You will  bind  all  of  You  as  if  made  by  all  of  You.\n- c. If You have mortgaged, pledged or hypothecated Your Home Building  and/or  Home  Contents  with  a  Bank,  the  Policy Schedule will  show  an  \n'Agreed Bank Clause' and the name of such Bank. The terms and conditions of this arrangement will be added to this Policy as an  additional  clause.  \n3. The  Policy  Schedule: The  Policy  Schedule  is  an  important document about Your insurance cover. It contains:\n- a. Your persona

In [72]:
doc1 = final_chunks[2]
print(doc1.page_content)

print(len(doc1.page_content))

- a. Your personal details,
- b. the  Policy  Period,
- c. the  description  of  Your  Insured  Property,
- d. the total  Sum  Insured,  the  Sum  Insured  for  each  cover  or  item covered, and any limits and sub-limits,
- e.  the insurance covers You have purchased,
- f. the  premium  You  have  paid  for  these  insurance  covers,
- g. add-on covers  opted  by  You,
- h. other  important  and  relevant  aspects  and  information.
4. Special meaning of certain words : Words stated in the table below have a special meaning throughout this Policy, the Policy Schedule  and  Endorsements.  
These  words  with  special  meaning  are  stated  in  the  Policy  with  the first  letter  in  capitals.  
Accident means sudden, unforeseen, and involuntary event caused by external, visible, and violent means  
Ambulance A road vehicle operated by a licenced/authorised service provider and equipped for the transport and paramedical treatment of the person requiring medical attention.
987


In [73]:
# Cell 7: Verification (Check Metadata Inheritance)
# Let's verify that a small chunk still knows it came from "Coverage A"

sample_chunk = final_chunks[0]

print("-" * 60)
print("FINAL CHUNK PREVIEW")
print("-" * 60)
print(f"Content Length: {len(sample_chunk.page_content)} chars")
print("Content Preview:")
print(sample_chunk.page_content[:200] + "...")
print("-" * 60)
print("METADATA (Inherited):")
print(sample_chunk.metadata)
print("-" * 60)

# Check
if len(sample_chunk.page_content) <= chunk_size + 50: # small buffer
    print("✅ Size Check Passed")
else:
    print("❌ Size Check Failed")


------------------------------------------------------------
FINAL CHUNK PREVIEW
------------------------------------------------------------
Content Length: 878 chars
Content Preview:
You chose this Simple Home Insurance Policy and applied to Us for insurance covers of Your choice. You paid Us the premium and gave Us  information  about  Yourself,  Your  Home  Building  and  Home C...
------------------------------------------------------------
METADATA (Inherited):
{'Header 2': 'SIMPLE HOME INSURANCE POLICY POLICY WORDING'}
------------------------------------------------------------
✅ Size Check Passed


In [75]:
# Cell 8: Initialize AWS Bedrock Embeddings (Authentication Fix)
import boto3
import os
from langchain_aws import BedrockEmbeddings
from dotenv import load_dotenv

# 1. Force reload the .env file to ensure we have the latest keys
load_dotenv(override=True)

# 2. debug: Print PART of the key to verify it is loaded (DO NOT print the whole key)
access_key = os.getenv("AWS_ACCESS_KEY_ID")
if access_key:
    print(f"Loaded Access Key ending in: ...{access_key[-4:]}")
else:
    print("❌ ERROR: AWS_ACCESS_KEY_ID not found in .env file")

Loaded Access Key ending in: ...IS63


In [76]:
# 3. Setup the Bedrock Client with EXPLICIT credentials
# This forces boto3 to use the keys from your code/env, ignoring the global ~/.aws folder
boto3_session = boto3.Session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_DEFAULT_REGION", "us-east-1")
)

print(f"Connecting to AWS Bedrock in region: {os.getenv('AWS_DEFAULT_REGION')}...")

Connecting to AWS Bedrock in region: None...


In [79]:
# 4. Define the Embedding Model (Sticking to V2)
embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",  # We stick to v2, it is better/cheaper
    client=boto3_session.client("bedrock-runtime")
)


# 5. TEST: Generate an embedding
# We take the first chunk we created in the previous step
test_text = "my name is prasad"    #final_chunks[0].page_content

print("Generating embedding for test chunk...")
try:
    vector = embeddings.embed_query(test_text)
    print("✅ SUCCESS: Embedding generated!")
    print(f"Vector Dimensions of amazon.titan-embed-text-v2:0 is --->: {len(vector)}")
    # Titan v2 defaults to 1024 dimensions
except Exception as e:
    print("❌ ERROR: Could not generate embeddings.")
    print(e)
    print("\nTroubleshooting:")
    print("1. Double check the AWS_ACCESS_KEY_ID in your .env file.")
    print("2. Ensure this IAM User has 'Bedrock Full Access' permissions.")

Generating embedding for test chunk...


2025-12-10 19:28:57,445 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '0c4857aa-48d3-4dec-850d-0668cfdc29a3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 10 Dec 2025 13:58:57 GMT', 'content-type': 'application/json', 'content-length': '43412', 'connection': 'keep-alive', 'x-amzn-requestid': '0c4857aa-48d3-4dec-850d-0668cfdc29a3', 'x-amzn-bedrock-invocation-latency': '97', 'x-amzn-bedrock-input-token-count': '6'}, 'RetryAttempts': 0}


✅ SUCCESS: Embedding generated!
Vector Dimensions of amazon.titan-embed-text-v2:0 is --->: 1024


In [52]:
# Cell 8.5: Debug & Reload Credentials
import os
from dotenv import load_dotenv

# 1. Force reload .env
load_dotenv(override=True)

# 2. Check the key (Safely)
key = os.getenv("PINECONE_API_KEY")

if not key:
    print("❌ ERROR: Key is EMPTY. Check .env file.")
elif key.startswith("pcsk_"):
    print(f"✅ Key loaded! Starts with: {key[:7]}... (Looks correct)")
else:
    print(f"⚠️ WARNING: Key format looks weird. It starts with: {key[:5]}...")
    print("Pinecone keys usually start with 'pcsk_'")

✅ Key loaded! Starts with: pcsk_2S... (Looks correct)


In [82]:
# Cell 9: Pinecone Setup & Ingestion
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# 1. Initialize Connection
pc_api_key = os.getenv("PINECONE_API_KEY")
if not pc_api_key:
    raise ValueError("❌ PINECONE_API_KEY not found in .env")

pc = Pinecone(api_key=pc_api_key)
print("✅ Pinecone Initialize Connection done succsesfully")

✅ Pinecone Initialize Connection done succsesfully


In [ ]:
# 2. Define Index Specification
index_name = "sbi-home-insurance-rag-test"
dimension = 1024  # Must match Amazon Titan v2 output

# Check if index exists, if not, create it
existing_indexes = [index.name for index in pc.list_indexes()]

if index_name not in existing_indexes:
    print(f"Index '{index_name}' not found. Creating new Serverless index...")
    try:
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric="cosine", # Titan works well with Cosine Similarity
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
        print("Index creating... waiting for initialization (approx 10-20s)")
        time.sleep(20) # Give it time to spin up
    except Exception as e:
        print(f"❌ Error creating index: {e}")
else:
    print(f"Index '{index_name}' already exists. Ready to use.")

In [ ]:
# 3. Connect to the Index via LangChain
# This wrapper handles the "Embedding -> Vector -> Upload" loop automatically
print(f"Uploading {len(final_chunks)} chunks to Pinecone...")

try:
    docsearch = PineconeVectorStore.from_documents(
        documents=final_chunks,
        embedding=embeddings, # We pass the Bedrock object we created in Cell 8
        index_name=index_name
    )
    print("✅ SUCCESS: Documents uploaded to Pinecone!")
    
except Exception as e:
    print("❌ ERROR during upload:")
    print(e)

In [55]:
# Cell 9: Pinecone Setup & Ingestion
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# 1. Initialize Connection
pc_api_key = os.getenv("PINECONE_API_KEY")
if not pc_api_key:
    raise ValueError("❌ PINECONE_API_KEY not found in .env")

pc = Pinecone(api_key=pc_api_key)

# 2. Define Index Specification
index_name = "sbi-home-insurance-rag-test"
dimension = 1024  # Must match Amazon Titan v2 output

# Check if index exists, if not, create it
existing_indexes = [index.name for index in pc.list_indexes()]

if index_name not in existing_indexes:
    print(f"Index '{index_name}' not found. Creating new Serverless index...")
    try:
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric="cosine", # Titan works well with Cosine Similarity
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
        print("Index creating... waiting for initialization (approx 10-20s)")
        time.sleep(20) # Give it time to spin up
    except Exception as e:
        print(f"❌ Error creating index: {e}")
else:
    print(f"Index '{index_name}' already exists. Ready to use.")

# 3. Connect to the Index via LangChain
# This wrapper handles the "Embedding -> Vector -> Upload" loop automatically
print(f"Uploading {len(final_chunks)} chunks to Pinecone...")

try:
    docsearch = PineconeVectorStore.from_documents(
        documents=final_chunks,
        embedding=embeddings, # We pass the Bedrock object we created in Cell 8
        index_name=index_name
    )
    print("✅ SUCCESS: Documents uploaded to Pinecone!")
    
except Exception as e:
    print("❌ ERROR during upload:")
    print(e)

Index 'sbi-home-insurance-rag-test' not found. Creating new Serverless index...
Index creating... waiting for initialization (approx 10-20s)
Uploading 212 chunks to Pinecone...


2025-12-06 12:56:58,739 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'f94e9b9f-2199-4dd9-916e-6757cb98a3c5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 07:26:57 GMT', 'content-type': 'application/json', 'content-length': '43268', 'connection': 'keep-alive', 'x-amzn-requestid': 'f94e9b9f-2199-4dd9-916e-6757cb98a3c5', 'x-amzn-bedrock-invocation-latency': '89', 'x-amzn-bedrock-input-token-count': '202'}, 'RetryAttempts': 0}
2025-12-06 12:57:01,043 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '5b80e76c-a780-4f7a-b9dc-5d93b0f9462f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 07:27:00 GMT', 'content-type': 'application/json', 'content-length': '43372', 'connection': 'keep-alive', 'x-amzn-requestid': '5b80e76c-a780-4f7a-b9dc-5d93b0f9462f', 'x-amzn-bedrock-invocation-latency': '87', 'x-amzn-bedrock-input-token-count': '250'}, 'RetryAttempts': 0}


✅ SUCCESS: Documents uploaded to Pinecone!


In [68]:
# Cell 10: Simple Retrieval Test
# Let's ask a question relevant to your policy PDF to see if it finds the right chunk.

query = "What is the name of the bank account holder and also provide me bank details of Siya Sharma?" 

print(f"Testing Retrieval for query: '{query}'")
print("-" * 50)

# Retrieve top 3 results
results = docsearch.similarity_search(query, k=10)

for i, doc in enumerate(results):
    print(f"\nRESULT #{i+1}")
    print(f"Source Header: {doc.metadata.get('Header 2', 'Unknown Section')}")
    print(f"Content snippet: {doc.page_content[:200]}...")

Testing Retrieval for query: 'What is the name of the bank account holder and also provide me bank details of Siya Sharma?'
--------------------------------------------------


2025-12-06 13:34:00,948 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'b535f74b-5d53-4f3a-8397-3afe9e017f6f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:03:58 GMT', 'content-type': 'application/json', 'content-length': '43395', 'connection': 'keep-alive', 'x-amzn-requestid': 'b535f74b-5d53-4f3a-8397-3afe9e017f6f', 'x-amzn-bedrock-invocation-latency': '85', 'x-amzn-bedrock-input-token-count': '21'}, 'RetryAttempts': 0}



RESULT #1
Source Header: Jewellery
Content snippet: Articles  of  personal  adornment  containing gemstones, silver, gold, platinum, or other precious metals.  
SBI General Insurance Company Limited, Corporate &amp; Registered Office: Fulcrum Building,...

RESULT #2
Source Header: Stage 4
Content snippet: You may approach the nearest Insurance Ombudsman for resolution of the grievance. Please refer the Annexure I for more information on ombudsman offices &amp; contact information.  
SBI General Insuran...

RESULT #3
Source Header: For the  purpose  of  this  Benefit,
Content snippet: - v. We will not make any payment under this Benefit if We have already paid or accepted any claims under Sections of Accidental Death or Permanent Total Disablement in respect of the Insured Person a...

RESULT #4
Source Header: A. What  We  Cover
Content snippet: SBI General Insurance Company Limited, Corporate &amp; Registered Office: Fulcrum Building, 9th Floor, A &amp; B Wing, Sahar Road, Andheri (Ea

In [69]:
# Cell 11: Initialize LLM (Llama 3.1 on Bedrock)
from langchain_aws import ChatBedrock
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. Define the LLM
# We use Llama 3.1 70B Instruct for high-quality reasoning
llm = ChatBedrock(
    model_id="meta.llama3-70b-instruct-v1:0", 
    client=boto3_session.client("bedrock-runtime"),
    model_kwargs={"temperature": 0.1, "max_gen_len": 512}
)

# 2. Define the Prompt (The "Instructions")
# We strictly tell the LLM to use ONLY the provided context.
prompt_template = """
You are an expert Insurance Assistant. Use the following pieces of retrieved context to answer the question.
If the answer is not in the context, just say that you don't know. Do not try to make up an answer.

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

prompt = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

# 3. Define the Chain (The Workflow)
# Retrieval -> Format Context -> Prompt -> LLM -> Output
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {"context": docsearch.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ LLM Pipeline Ready.")

✅ LLM Pipeline Ready.


In [ ]:
# Cell 12: Final Test (End-to-End RAG)
query = """According to the table in Section 1 - Fire and Allied Perils, what specific exclusions apply to loss or damage caused by:

Subsidence, landslide, or rockslide?

Riot, strikes, or malicious damages?"""
#"give me total transaction history on 20/07/2021 including there payment mode also"
#"what is the Total Debit Amount and Closing Balance"
#"What is the name of the bank account holder and also provide me bank details of Siya Sharma?"

print(f"QUESTION: {query}")
print("-" * 50)

# Run the chain
response = rag_chain.invoke(query)

print("AI ANSWER:")
print(response)

QUESTION: According to the table in Section 1 – Fire and Allied Perils, what specific exclusions apply to loss or damage caused by:

Subsidence, landslide, or rockslide?

Riot, strikes, or malicious damages?
--------------------------------------------------


2025-12-06 13:34:29,613 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '2eb86f39-c9f5-4eca-a80e-a6e1cc5b290d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:04:26 GMT', 'content-type': 'application/json', 'content-length': '43337', 'connection': 'keep-alive', 'x-amzn-requestid': '2eb86f39-c9f5-4eca-a80e-a6e1cc5b290d', 'x-amzn-bedrock-invocation-latency': '83', 'x-amzn-bedrock-input-token-count': '47'}, 'RetryAttempts': 0}
2025-12-06 13:34:31,415 - INFO - Using Bedrock Invoke API to generate response


AI ANSWER:
According to the provided context, the answer is:

Riot, strikes, or malicious damages: Exclusions do not apply. Instead, there are specific conditions mentioned in Section 10, which states that the policy covers loss or damage caused by riot, strikes, or malicious damages unless it is caused by a. temporary or permanent dispossession, confiscation, commandeering, requisition or destruction by order of the government or any lawful authority, or b. temporary or permanent dispossession of Your Home by unlawful occupation by any person.

I don't know the specific exclusions for subsidence, landslide, or rockslide as it is not mentioned in the provided context.


In [71]:
# Cell 13: Diagnostic - Find the "Missing" Chunk
search_term = "Subsidence"
found_count = 0

print(f"🔍 Scanning all {len(final_chunks)} chunks for the word '{search_term}'...\n")

for i, chunk in enumerate(final_chunks):
    if search_term.lower() in chunk.page_content.lower():
        found_count += 1
        print(f"✅ FOUND in Chunk #{i}")
        print("-" * 30)
        print(f"Metadata: {chunk.metadata}")
        print("-" * 30)
        print("CONTENT PREVIEW:")
        print(chunk.page_content[:300] + "...") # Print first 300 chars
        print("\n" + "=" * 50 + "\n")

if found_count == 0:
    print(f"❌ CRITICAL ERROR: The word '{search_term}' does NOT exist in any chunk.")
    print("This means Docling failed to read that part of the PDF.")
else:
    print(f"✅ VERIFIED: The data exists in {found_count} chunks. The issue is purely Retrieval (Vector Search).")

🔍 Scanning all 212 chunks for the word 'Subsidence'...

✅ FOUND in Chunk #57
------------------------------
Metadata: {'Header 2': 'SECTION 1 -FIRE  AND  ALLIED  PERILS', 'text': '| 5. Storm, Cyclone, Typhoon, Tempest, Hurricane, Tornado, Tsunami, Flood and Inundation                                                                       | -                                                                                                                                                                                                                                                                                                                                        |\n| 6. SubsidenceofthelandonwhichYourHomeBuildingstands, Landslide, Rockslide                                                                                    | caused by a. normal cracking, settlement or bedding down of newstructures, b. the settlement or movement of made up ground, c. coastal or river erosion, d. defective 

In [72]:
# Cell 14: Hybrid Search Setup (BM25 + Dense)
import time
from pinecone import Pinecone, ServerlessSpec
from pinecone_text.sparse import BM25Encoder

# 1. Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "sbi-home-insurance-rag-hybrid"  # New name for the hybrid index

# 2. Setup BM25 Encoder (The "Keyword" Brain)
# We must "teach" the encoder about our document's vocabulary.
print("Fitting BM25 Encoder on your chunks...")
bm25 = BM25Encoder()
chunk_texts = [chunk.page_content for chunk in final_chunks]
bm25.fit(chunk_texts)

# Save the encoder to a json file (Production Best Practice: You load this later instead of refitting)
bm25.dump("bm25_values.json")
print("✅ BM25 Encoder fitted and saved.")

# 3. Create a Hybrid-Compatible Index (Dotproduct)
existing_indexes = [index.name for index in pc.list_indexes()]

if index_name not in existing_indexes:
    print(f"Creating new Hybrid Index: {index_name}...")
    pc.create_index(
        name=index_name,
        dimension=1024, # Titan v2
        metric="dotproduct", # REQUIRED for Hybrid Search
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    time.sleep(10) # Wait for init
else:
    print(f"Index {index_name} exists.")

index = pc.Index(index_name)

# 4. Upload Data (Dense + Sparse)
# We have to manually construct the vectors because LangChain's simple wrapper doesn't do Hybrid upsert easily yet.
print(f"Generating Hybrid Vectors for {len(final_chunks)} chunks...")

vectors_to_upsert = []

for i, chunk in enumerate(final_chunks):
    # A. Generate Dense Vector (Titan)
    dense_vector = embeddings.embed_query(chunk.page_content)
    
    # B. Generate Sparse Vector (BM25)
    sparse_vector = bm25.encode_documents(chunk.page_content)
    
    # C. Create Metadata (Keep the text so we can retrieve it!)
    metadata = {
        "text": chunk.page_content,
        "header_1": chunk.metadata.get("Header 1", ""),
        "header_2": chunk.metadata.get("Header 2", ""),
        "source": chunk.metadata.get("source", "unknown")
    }
    
    # D. Bundle it
    # ID must be a string
    vectors_to_upsert.append({
        "id": str(i),
        "values": dense_vector,
        "sparse_values": sparse_vector,
        "metadata": metadata
    })

# Batch Upload (Pinecone likes batches of 100)
batch_size = 50
print("Uploading to Pinecone...")

for i in range(0, len(vectors_to_upsert), batch_size):
    batch = vectors_to_upsert[i : i + batch_size]
    index.upsert(vectors=batch)
    print(f"Uploaded batch {i} - {i+batch_size}")

print("✅ Hybrid Index Ready!")

Fitting BM25 Encoder on your chunks...


100%|██████████| 212/212 [00:00<00:00, 221.29it/s]


✅ BM25 Encoder fitted and saved.
Creating new Hybrid Index: sbi-home-insurance-rag-hybrid...
Generating Hybrid Vectors for 212 chunks...


2025-12-06 13:42:30,275 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '874a5ade-8afc-4a07-8bad-eed7fbf8b612', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:12:29 GMT', 'content-type': 'application/json', 'content-length': '43268', 'connection': 'keep-alive', 'x-amzn-requestid': '874a5ade-8afc-4a07-8bad-eed7fbf8b612', 'x-amzn-bedrock-invocation-latency': '78', 'x-amzn-bedrock-input-token-count': '202'}, 'RetryAttempts': 0}
2025-12-06 13:42:34,189 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '0718e6b2-3951-4f27-a6ff-0a64b8d0f4dc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:12:33 GMT', 'content-type': 'application/json', 'content-length': '43372', 'connection': 'keep-alive', 'x-amzn-requestid': '0718e6b2-3951-4f27-a6ff-0a64b8d0f4dc', 'x-amzn-bedrock-invocation-latency': '88', 'x-amzn-bedrock-input-token-count': '250'}, 'RetryAttempts': 0}


Uploading to Pinecone...
Uploaded batch 0 - 50
Uploaded batch 50 - 100
Uploaded batch 100 - 150
Uploaded batch 150 - 200
Uploaded batch 200 - 250
✅ Hybrid Index Ready!


In [74]:
# Cell 15: Hybrid Retrieval Test
def hybrid_search(query, top_k=5, alpha=0.4):
    # 1. Generate query embeddings
    dense_vec = embeddings.embed_query(query)
    sparse_vec = bm25.encode_queries(query)
    
    # 2. Search Pinecone
    results = index.query(
        vector=dense_vec,
        sparse_vector=sparse_vec,
        top_k=top_k,
        include_metadata=True,
        hybrid_alpha=alpha # The magic lever
    )
    
    return results

# TEST
query = """ According to the table in Section 1 – Fire and Allied Perils, what specific exclusions apply to loss or damage caused by:

Subsidence, landslide, or rockslide?

Riot, strikes, or malicious damages?"""
#"What specific exclusions apply to loss caused by Subsidence, landslide, or rockslide?"
print(f"Testing Hybrid Search for: '{query}'")

results = hybrid_search(query, top_k=5)

for match in results['matches']:
    print(f"\nScore: {match['score']:.4f}")
    print(f"Content: {match['metadata']['text'][:200]}...")

Testing Hybrid Search for: ' According to the table in Section 1 – Fire and Allied Perils, what specific exclusions apply to loss or damage caused by:

Subsidence, landslide, or rockslide?

Riot, strikes, or malicious damages?'


2025-12-06 14:00:24,534 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '089e61e1-28b5-4c30-9c67-c076ae692fc7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:30:20 GMT', 'content-type': 'application/json', 'content-length': '43381', 'connection': 'keep-alive', 'x-amzn-requestid': '089e61e1-28b5-4c30-9c67-c076ae692fc7', 'x-amzn-bedrock-invocation-latency': '79', 'x-amzn-bedrock-input-token-count': '47'}, 'RetryAttempts': 0}



Score: 0.7141
Content: | 10. Riot, Strikes, Malicious Damages                                                                             | caused by a. temporary or permanent dispossession, confiscation, commandeering, req...

Score: 0.5615
Content: 1. Loss or damage to bullion or unset precious stones, manuscripts, plans, drawings, securities, obligations or documents of any kind, coins  or  paper  money,  cheques,  vehicles,  and  explosive sub...

Score: 0.5011
Content: 20. revolution,  insurrection,  conspiracy,  military  or  usurped power.
12.  Loss or Damage directly or indirectly caused by or contributed to  by  or  arising  from  ionizing  radiations  contamina...

Score: 0.4930
Content: - a.  As soon as any loss or damage occurs to the Insured Property, You  must  give  immediate  report to appropriate legal authorities. For example, You must report to the fire brigade of the local a...

Score: 0.4856
Content: | 5. Storm, Cyclone, Typhoon, Tempest, Hurricane, Tornado, Tsuna

In [77]:
# Cell 16: Hybrid RAG Chain (The Final Pipeline)
from langchain_core.retrievers import BaseRetriever
from typing import List, Any  # <-- Added 'Any' here
from langchain_core.documents import Document

# 1. Define a Custom Retriever for Hybrid Search
class PineconeHybridRetriever(BaseRetriever):
    index: Any
    embeddings: Any
    bm25: Any
    top_k: int = 5
    alpha: float = 0.4

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # A. Generate vectors
        dense_vec = self.embeddings.embed_query(query)
        sparse_vec = self.bm25.encode_queries(query)
        
        # B. Search Pinecone
        results = self.index.query(
            vector=dense_vec,
            sparse_vector=sparse_vec,
            top_k=self.top_k,
            include_metadata=True,
            hybrid_alpha=self.alpha
        )
        
        # C. Convert back to LangChain Documents
        final_docs = []
        for match in results['matches']:
            text = match['metadata']['text']
            # Remove text from metadata to save space
            metadata = {k:v for k,v in match['metadata'].items() if k != 'text'}
            final_docs.append(Document(page_content=text, metadata=metadata))
            
        return final_docs

# 2. Initialize the Retriever
hybrid_retriever = PineconeHybridRetriever(
    index=index, 
    embeddings=embeddings, 
    bm25=bm25
)

# 3. Re-build the Chain with the NEW Retriever
rag_chain_hybrid = (
    {"context": hybrid_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Hybrid RAG Pipeline Connected.")

# 4. THE FINAL TEST
query = """ According to the table in Section 1 – Fire and Allied Perils, what specific exclusions apply to loss or damage caused by:

Subsidence, landslide, or rockslide?

Riot, strikes, or malicious damages?"""

print(f"\nQUESTION: {query}")
print("-" * 50)
response = rag_chain_hybrid.invoke(query)
print("AI ANSWER:")
print(response)

✅ Hybrid RAG Pipeline Connected.

QUESTION:  According to the table in Section 1 – Fire and Allied Perils, what specific exclusions apply to loss or damage caused by:

Subsidence, landslide, or rockslide?

Riot, strikes, or malicious damages?
--------------------------------------------------


2025-12-06 14:08:04,385 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'f6e627da-22d1-49ca-8908-df45e34dda57', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:38:02 GMT', 'content-type': 'application/json', 'content-length': '43381', 'connection': 'keep-alive', 'x-amzn-requestid': 'f6e627da-22d1-49ca-8908-df45e34dda57', 'x-amzn-bedrock-invocation-latency': '81', 'x-amzn-bedrock-input-token-count': '47'}, 'RetryAttempts': 0}
2025-12-06 14:08:09,575 - INFO - Using Bedrock Invoke API to generate response


AI ANSWER:
Based on the provided context, here are the answers to the questions:

1. According to the table in Section 1 – Fire and Allied Perils, what specific exclusions apply to loss or damage caused by Subsidence, landslide, or rockslide?

The specific exclusions that apply are:
a. normal cracking, settlement or bedding down of new structures,
b. the settlement or movement of made up ground,
c. coastal or river erosion,
d. defective design or workmanship or use of defective materials, or demolition, construction, structural alterations or repair of any property, or groundworks or excavations.

2. According to the table in Section 1 – Fire and Allied Perils, what specific exclusions apply to loss or damage caused by Riot, strikes, or malicious damages?

The specific exclusions that apply are:
a. temporary or permanent dispossession, confiscation, commandeering, requisition or destruction by order of the government or any lawful authority,
b. temporary or permanent dispossession of Y

In [78]:
# Cell 19: Define Evaluation Dataset
from datasets import Dataset

# 1. The "Golden" Questions and Answers
# In a real project, human experts would write these.
test_questions = [
    "What specific exclusions apply to loss caused by Subsidence?", 
    "What are the exclusions for Riot, strikes, or malicious damages?",
    "What is the deductible for Personal Property?" 
]

# 2. The "Ground Truths" (The correct answers we expect)
ground_truths = [
    ["Normal cracking, settlement of new structures, movement of made up ground, coastal erosion, defective design or workmanship."],
    ["Temporary or permanent dispossession by government order, or unlawful occupation by any person."],
    ["$1,000"]
]

# 3. Generate Answers using YOUR Pipeline
print("Generating answers from your RAG pipeline...")

answers = []
contexts = []

for query in test_questions:
    print(f"Asking: {query}")
    
    # Run your Hybrid Chain
    response_text = rag_chain_hybrid.invoke(query)
    answers.append(response_text)
    
    # We also need to save the "Context" the LLM saw, so Ragas can check if it was used.
    # We retrieve the docs manually to save them
    docs = hybrid_retriever.invoke(query)
    context_text = [d.page_content for d in docs]
    contexts.append(context_text)

# 4. Create the Dataset Object
data = {
    "question": test_questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(data)

print("\n✅ Evaluation Dataset Created!")
print(f"Sample Answer 1: {answers[0][:100]}...")

Generating answers from your RAG pipeline...
Asking: What specific exclusions apply to loss caused by Subsidence?


2025-12-06 14:18:31,708 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '07b2c425-2ee8-47c6-91f4-71f63265b301', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:48:31 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-alive', 'x-amzn-requestid': '07b2c425-2ee8-47c6-91f4-71f63265b301', 'x-amzn-bedrock-invocation-latency': '81', 'x-amzn-bedrock-input-token-count': '14'}, 'RetryAttempts': 0}
2025-12-06 14:18:38,010 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 14:18:40,010 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '8923182e-2a21-4fa6-b14e-e9f4f06a32f3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:48:39 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-alive', 'x-amzn-requestid': '8923182e-2a21-4fa6-b14e-e9f4f06a32f3', 'x-amzn-bedrock-invocation-la

Asking: What are the exclusions for Riot, strikes, or malicious damages?


2025-12-06 14:18:42,693 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '48ef4303-a8fc-4b01-8f70-21c102d191ae', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:48:42 GMT', 'content-type': 'application/json', 'content-length': '43315', 'connection': 'keep-alive', 'x-amzn-requestid': '48ef4303-a8fc-4b01-8f70-21c102d191ae', 'x-amzn-bedrock-invocation-latency': '85', 'x-amzn-bedrock-input-token-count': '15'}, 'RetryAttempts': 0}
2025-12-06 14:18:44,820 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 14:18:48,267 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '9e3207e7-90e1-4834-9661-49b0ab1f0f36', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:48:46 GMT', 'content-type': 'application/json', 'content-length': '43315', 'connection': 'keep-alive', 'x-amzn-requestid': '9e3207e7-90e1-4834-9661-49b0ab1f0f36', 'x-amzn-bedrock-invocation-la

Asking: What is the deductible for Personal Property?


2025-12-06 14:18:53,392 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'd3c129d0-b983-4939-b129-eea7292cf237', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:48:51 GMT', 'content-type': 'application/json', 'content-length': '43328', 'connection': 'keep-alive', 'x-amzn-requestid': 'd3c129d0-b983-4939-b129-eea7292cf237', 'x-amzn-bedrock-invocation-latency': '66', 'x-amzn-bedrock-input-token-count': '9'}, 'RetryAttempts': 0}
2025-12-06 14:18:56,409 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 14:19:01,709 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '24589b31-3298-4781-9deb-a11f203ebd8e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 08:48:59 GMT', 'content-type': 'application/json', 'content-length': '43328', 'connection': 'keep-alive', 'x-amzn-requestid': '24589b31-3298-4781-9deb-a11f203ebd8e', 'x-amzn-bedrock-invocation-lat


✅ Evaluation Dataset Created!
Sample Answer 1: I don't know. The context does not mention subsidence as a specific exclusion....


In [86]:
# Cell 20: Initialize Cohere Re-ranker (FIXED)
import json
import boto3
from typing import List
from langchain_core.documents import Document

class BedrockCohereReranker:
    def __init__(self, region_name="us-east-1"):
        self.client = boto3.client("bedrock-runtime", region_name=region_name)
        self.model_id = "cohere.rerank-v3-5:0"

    def rerank(self, query: str, docs: List[Document], top_n: int = 5) -> List[Document]:
        if not docs: return []
        
        # FIX: Send a simple list of strings, NOT a list of dictionaries
        documents_payload = [d.page_content for d in docs]
        
        request_body = {
            "query": query, 
            "documents": documents_payload, 
            "top_n": top_n, 
            "api_version": 2
        }

        try:
            response = self.client.invoke_model(modelId=self.model_id, body=json.dumps(request_body))
            response_body = json.loads(response['body'].read())
            results = response_body.get("results", [])
            
            # Map back to documents
            reranked_docs = []
            for res in results:
                # The 'index' field tells us which string in the list matched
                original_index = res["index"]
                doc = docs[original_index]
                
                # Keep metadata but update score
                doc.metadata["rerank_score"] = res["relevance_score"]
                reranked_docs.append(doc)
                
            return reranked_docs
        except Exception as e:
            print(f"Rerank Error: {e}")
            # Fallback: return original docs if rerank fails
            return docs[:top_n]

reranker = BedrockCohereReranker()
print("✅ Re-ranker Initialized (Fixed Payload).")

✅ Re-ranker Initialized (Fixed Payload).


In [89]:
# Cell 21 (FINAL ROBUST VERSION): Production Pipeline
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# 1. Force the retriever to cast a wide net (Top 25)
hybrid_retriever.top_k = 25 

# 2. Define a single "Smart Retrieval" function
# This function takes the User Query -> Returns the Best Context String
def intelligent_retrieval(query: str):
    # Step A: Hybrid Search (Get 25 candidates)
    # We use .invoke() to run the retriever
    raw_docs = hybrid_retriever.invoke(query)
    
    # Step B: Reranking (Filter to Top 5)
    # The reranker takes the query and the 25 docs
    top_5_docs = reranker.rerank(query, raw_docs, top_n=5)
    
    # Step C: Format for LLM
    context_text = "\n\n".join([d.page_content for d in top_5_docs])
    
    return context_text

# 3. Build the Final Chain
# logic: { context: get_smart_context(query), question: pass_query } -> prompt -> llm
rag_chain_final = (
    {
        "context": RunnableLambda(intelligent_retrieval), # Runs the function above
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Production Pipeline Repaired (Robust Version).")

# 4. Final Verification
test_query = "What specific exclusions apply to loss caused by Subsidence?"

print(f"\nQUESTION: {test_query}")
print("-" * 40)
# This should now return the clean answer
print(rag_chain_final.invoke(test_query))

✅ Production Pipeline Repaired (Robust Version).

QUESTION: What specific exclusions apply to loss caused by Subsidence?
----------------------------------------


2025-12-06 15:19:00,234 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '84b4f13b-858b-4608-b2b3-0ef873f14183', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 09:48:59 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-alive', 'x-amzn-requestid': '84b4f13b-858b-4608-b2b3-0ef873f14183', 'x-amzn-bedrock-invocation-latency': '98', 'x-amzn-bedrock-input-token-count': '14'}, 'RetryAttempts': 0}
2025-12-06 15:19:08,546 - INFO - Using Bedrock Invoke API to generate response


According to the context, the specific exclusions that apply to loss caused by Subsidence are:

a. normal cracking, settlement or bedding down of new structures,
b. the settlement or movement of made up ground,
c. coastal or river erosion,
d. defective design or workmanship or use of defective materials, or demolition, construction, structural alterations or repair of any property, or groundworks or excavations.


In [90]:
# Cell 23: Final "Golden Set" Evaluation
from datasets import Dataset
import pandas as pd

# 1. The Hard Questions
test_questions = [
    "What specific exclusions apply to loss caused by Subsidence?", 
    "What is the deductible for Personal Property?",
    "What are the specific exclusions for Riot, strikes, or malicious damages?"
]

# 2. Generate Answers using the NEW Production Pipeline
print("running final evaluation on 3 test questions...")
print("-" * 50)

results = []

for q in test_questions:
    print(f"Asking: {q}")
    try:
        # Run the robust chain
        answer = rag_chain_final.invoke(q)
        
        # Save result
        results.append({
            "Question": q,
            "AI Answer": answer,
            "Status": "✅ Generated"
        })
    except Exception as e:
        results.append({
            "Question": q,
            "AI Answer": f"ERROR: {e}",
            "Status": "❌ Failed"
        })

# 3. Display Results in a Clean Table
df = pd.DataFrame(results)

print("\n" + "="*50)
print("FINAL ACCURACY REPORT")
print("="*50)

# We print the full answers to verify details
for i, row in df.iterrows():
    print(f"\nQ{i+1}: {row['Question']}")
    print(f"A: {row['AI Answer'].strip()}")
    print("-" * 30)

running final evaluation on 3 test questions...
--------------------------------------------------
Asking: What specific exclusions apply to loss caused by Subsidence?


2025-12-06 15:20:35,037 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '2736152f-374c-4069-8bb8-bed186eb75a4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 09:50:33 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-alive', 'x-amzn-requestid': '2736152f-374c-4069-8bb8-bed186eb75a4', 'x-amzn-bedrock-invocation-latency': '86', 'x-amzn-bedrock-input-token-count': '14'}, 'RetryAttempts': 0}
2025-12-06 15:20:41,449 - INFO - Using Bedrock Invoke API to generate response


Asking: What is the deductible for Personal Property?


2025-12-06 15:20:45,311 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'c34ef811-454e-43ee-91a5-c7f5b12b5fb0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 09:50:44 GMT', 'content-type': 'application/json', 'content-length': '43328', 'connection': 'keep-alive', 'x-amzn-requestid': 'c34ef811-454e-43ee-91a5-c7f5b12b5fb0', 'x-amzn-bedrock-invocation-latency': '81', 'x-amzn-bedrock-input-token-count': '9'}, 'RetryAttempts': 0}
2025-12-06 15:20:49,344 - INFO - Using Bedrock Invoke API to generate response


Asking: What are the specific exclusions for Riot, strikes, or malicious damages?


2025-12-06 15:20:52,930 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '24706e24-5d52-4687-8414-19ed586f575f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 09:50:51 GMT', 'content-type': 'application/json', 'content-length': '43437', 'connection': 'keep-alive', 'x-amzn-requestid': '24706e24-5d52-4687-8414-19ed586f575f', 'x-amzn-bedrock-invocation-latency': '83', 'x-amzn-bedrock-input-token-count': '16'}, 'RetryAttempts': 0}
2025-12-06 15:20:56,552 - INFO - Using Bedrock Invoke API to generate response



FINAL ACCURACY REPORT

Q1: What specific exclusions apply to loss caused by Subsidence?
A: According to the context, the specific exclusions that apply to loss caused by Subsidence are:

* normal cracking, settlement or bedding down of new structures
* the settlement or movement of made up ground
* coastal or river erosion
* defective design or workmanship or use of defective materials
* demolition, construction, structural alterations or repair of any property
* groundworks or excavations.
------------------------------

Q2: What is the deductible for Personal Property?
A: The context does not mention a specific deductible for Personal Property. The deductibles mentioned are for Jewellery & Valuables (5% of the claim amount subject to a minimum of Rs 2500) and portable equipment's (5% of claim amount subject to a minimum of Rs 1000). Therefore, I don't know the deductible for Personal Property.
------------------------------

Q3: What are the specific exclusions for Riot, strikes, or

In [93]:
# Cell 24: Calculate Ragas Metrics (FIXED & SAFE MODE)
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.run_config import RunConfig  # <-- Import this to control speed
from datasets import Dataset

# 1. Define the Questions & Ground Truths
questions = [
    "What specific exclusions apply to loss caused by Subsidence?", 
    "What is the deductible for Personal Property?",
    "What are the specific exclusions for Riot, strikes, or malicious damages?"
]

ground_truths = [
    "Normal cracking, settlement of new structures, movement of made up ground, coastal erosion, defective design.",
    "The document does not state a specific deductible for 'Personal Property', only for Jewellery and Portables.",
    "Temporary or permanent dispossession by government order, or unlawful occupation by any person."
]

# 2. Generate Answers & Fetch Contexts (Manually)
print("Generating answers and fetching contexts...")

answers = []
contexts = []

for q in questions:
    # A. Get Answer from Chain
    print(f"Processing: {q}")
    answer = rag_chain_final.invoke(q)
    answers.append(answer)
    
    # B. Get Contexts (We manually run the retrieval + rerank logic here)
    # 1. Hybrid Search
    raw_docs = hybrid_retriever.invoke(q)
    # 2. Rerank (Same logic as production chain)
    reranked_docs = reranker.rerank(q, raw_docs, top_n=5)
    # 3. Extract text list
    context_list = [d.page_content for d in reranked_docs]
    contexts.append(context_list)

# 3. Build the Ragas Dataset
data_samples = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

ragas_dataset = Dataset.from_dict(data_samples)

# 4. Configure Ragas to use AWS Bedrock
ragas_llm = LangchainLLMWrapper(llm)
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# 5. Define SAFE Configuration (Slow & Steady)
# max_workers=1 ensures we send only 1 request at a time to Bedrock
safe_config = RunConfig(
    max_workers=1, 
    timeout=120,
    max_retries=3
)

# 6. Run Evaluation
print("\n👨‍⚖️ The AI Judges are voting... (Sequential Mode - Slower but Safer)")

results = evaluate(
    ragas_dataset,
    metrics=[
        faithfulness,       
        answer_relevancy,   
        context_precision  
    ],
    llm=ragas_llm,
    embeddings=ragas_embeddings,
    run_config=safe_config,  # <-- Apply the speed limit
    raise_exceptions=False   # Prevent crashing if one metric fails
)

# 7. Display Results
print("\n" + "="*50)
print("🏆 FINAL RAG REPORT CARD")
print("="*50)
print(results)

# Convert to pandas for display
df_results = results.to_pandas()

# Safety check: ensure columns exist before printing
if 'question' in df_results.columns:
    print("\nDetailed Breakdown:")
    print(df_results[['question', 'faithfulness', 'answer_relevancy']].to_markdown())
else:
    print("\nFull Results Table:")
    print(df_results)

Generating answers and fetching contexts...
Processing: What specific exclusions apply to loss caused by Subsidence?


2025-12-06 15:33:52,154 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'e9dcff38-3c82-4312-8bee-4c7cfe3ef91e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 10:03:48 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-alive', 'x-amzn-requestid': 'e9dcff38-3c82-4312-8bee-4c7cfe3ef91e', 'x-amzn-bedrock-invocation-latency': '78', 'x-amzn-bedrock-input-token-count': '14'}, 'RetryAttempts': 0}
2025-12-06 15:33:59,756 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 15:34:05,468 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'f0c5d76b-79d0-409d-9379-90e07b7868d7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 10:04:02 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-alive', 'x-amzn-requestid': 'f0c5d76b-79d0-409d-9379-90e07b7868d7', 'x-amzn-bedrock-invocation-la

Processing: What is the deductible for Personal Property?


2025-12-06 15:34:09,801 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '31b66568-ff0f-4cf1-94d7-99272271a765', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 10:04:08 GMT', 'content-type': 'application/json', 'content-length': '43328', 'connection': 'keep-alive', 'x-amzn-requestid': '31b66568-ff0f-4cf1-94d7-99272271a765', 'x-amzn-bedrock-invocation-latency': '85', 'x-amzn-bedrock-input-token-count': '9'}, 'RetryAttempts': 0}
2025-12-06 15:34:12,554 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 15:34:15,386 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '8423f4aa-e3a1-41d3-93e5-b930025db195', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 10:04:14 GMT', 'content-type': 'application/json', 'content-length': '43328', 'connection': 'keep-alive', 'x-amzn-requestid': '8423f4aa-e3a1-41d3-93e5-b930025db195', 'x-amzn-bedrock-invocation-lat

Processing: What are the specific exclusions for Riot, strikes, or malicious damages?


2025-12-06 15:34:24,212 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': 'f438e44d-52cc-45eb-89a0-4eded44d3ae9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 10:04:23 GMT', 'content-type': 'application/json', 'content-length': '43437', 'connection': 'keep-alive', 'x-amzn-requestid': 'f438e44d-52cc-45eb-89a0-4eded44d3ae9', 'x-amzn-bedrock-invocation-latency': '74', 'x-amzn-bedrock-input-token-count': '16'}, 'RetryAttempts': 0}
2025-12-06 15:34:42,136 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 15:34:46,089 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '1952d948-910e-48aa-a45a-bb7bd77b3bfa', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 10:04:45 GMT', 'content-type': 'application/json', 'content-length': '43437', 'connection': 'keep-alive', 'x-amzn-requestid': '1952d948-910e-48aa-a45a-bb7bd77b3bfa', 'x-amzn-bedrock-invocation-la


👨‍⚖️ The AI Judges are voting... (Sequential Mode - Slower but Safer)


Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]2025-12-06 15:35:02,180 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 15:35:12,759 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 15:35:25,516 - ERROR - Exception raised in Job[0]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Evaluating:  11%|█         | 1/9 [00:23<03:06, 23.35s/it]2025-12-06 15:35:25,533 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 15:35:25,534 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 15:35:25,535 - INFO - Using Bedrock Invoke API to generate response
2025-12-06 15:35:32,449 - INFO - Successfully invoked model amazon.titan-embed-text-v2:0. ResponseMetadata: {'RequestId': '025fa482-8c6f-4e08-b60c-0266682af9f9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 06 Dec 2025 10:05:30 GMT', 'content-type': 'application/json', 'content-length': '43375', 'connection': 'keep-al


🏆 FINAL RAG REPORT CARD
{'faithfulness': 1.0000, 'answer_relevancy': 0.9948, 'context_precision': 0.9625}

Full Results Table:
                                          user_input  \
0  What specific exclusions apply to loss caused ...   
1      What is the deductible for Personal Property?   
2  What are the specific exclusions for Riot, str...   

                                  retrieved_contexts  \
0  [| 5. Storm, Cyclone, Typhoon, Tempest, Hurric...   
1  [If You make a claim under this Policy which i...   
2  [| 10. Riot, Strikes, Malicious Damages       ...   

                                            response  \
0  According to the context, the specific exclusi...   
1  The context does not mention the deductible fo...   
2  According to the context, the specific exclusi...   

                                           reference  faithfulness  \
0  Normal cracking, settlement of new structures,...           NaN   
1  The document does not state a specific deducti...     